In [133]:
import numpy as np
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [134]:
def make_chains(mat, names, k, key):
    '''Передаём key, чтобы словарь поиска содеражал такие ключи'''
    def dfs(que, mat, v, k_now, k_max, chainsDict, names):
        if k_now == k_max:
            name_in_num = list(que)
            name_in_num.sort()
            name_in_num = list(map(lambda i: names[i], name_in_num))
            kk = reduce(lambda i, j: i + j, name_in_num)
            chainsDict.setdefault(kk, 0)
            chainsDict[kk] += 1
            return
        
        for i in range(0, mat.shape[0]):
            if mat[v, i] == 1 and not i in que:
                que.append(i)
                dfs(que, mat, i, k_now + 1, k_max, chainsDict, names)
                del(que[k_now])
    
    
            
    chainsDict = dict.fromkeys(key, 0)
    
    for i in range(mat.shape[0]):#Запускаем обход в глубину k из каждой вершины
        que = []
        que.append(i)
        
        dfs(que, mat, i, 1, k, chainsDict, names)
        del(que[0])
        
        
    for i in chainsDict.keys(): #Каждую цепочку посчитали два раза
        chainsDict[i] //= 2
    return chainsDict
        

In [135]:
def getAllKChainsNameInFile(filename, k): #k-размер цепочек
    all_chains_names = []
    ctr = 0
    with  open(filename, "r") as f:
        
        
        for i in f:
            ctr += 1
            mol_atoms = []
            mol_name = i.split()[1]
            num_elems = int(i.split()[0])
            

            matrix = np.zeros((num_elems, num_elems))
            for i in range(num_elems):
                
                line = next(f) #Берём i линию в молекуле
                
                mol_atoms.append(line.split()[0]) #По первому индексу хранятся названия, которые пихаем в лист

                it = 3 #С третьего элемента в строке начинаются связи
                while line.split()[it] != "00":
                    matrix[i, int(line.split()[it][:-1]) - 1] = 1 #int(line.split()[it][-1]) Если захотим тип связи
                    matrix[int(line.split()[it][:-1]) - 1, i] = 1
                    it += 1
            res = make_chains(matrix, mol_atoms, k, [])
            all_chains_names += res.keys()
#             print(matrix, "\n", res)
#             print()
#             print()
        
    returnList = list(np.unique(all_chains_names))
    returnList.sort()
    return returnList, ctr

In [136]:
def getMatrix(filename, k, keys, high):
    res_matr = np.zeros((high, len(keys)))
    dic = dict.fromkeys(keys, 0)
    cnt = 0
    with open(filename, "r") as f:
        for i in f:
            
            mol_atoms = []
            num_elems = int(i.split()[0])
            matrix = np.zeros((num_elems, num_elems))
            for i in range(num_elems):
                line = next(f) #Берём i линию в молекуле
                mol_atoms.append(line.split()[0]) #По первому индексу хранятся названия, которые пихаем в лист

                it = 3 #С третьего элемента в строке начинаются связи
                while line.split()[it] != "00":
                    matrix[i, int(line.split()[it][:-1]) - 1] = 1 #int(line.split()[it][-1]) Если захотим тип связи
                    matrix[int(line.split()[it][:-1]) - 1, i] = 1
                    it += 1
            res = make_chains(matrix, mol_atoms, k, dic.keys())
            
            res_matr[cnt,:] = list(res.values())
            cnt += 1
            
    return res_matr

In [137]:
y = []
file_y = "ALKAN.SET"

In [138]:
with open(file_y, "r") as f:
    for i in f:
        splitString = i.split()
        if len(splitString) > 0 and splitString[0] == 'A1':
            y.append(float(splitString[1]))

In [139]:
filename = "ALKAN.STR"
len(y)

74

In [289]:
chainLen = 2

In [290]:
names, numMolec = getAllKChainsNameInFile(filename, chainLen)

In [291]:
Matr = getMatrix(filename, chainLen, names, numMolec)

In [292]:
# for i in range(Matr.shape[0]):
#     for j in range(Matr.shape[1]):
#         print(Matr[i,j], end=" ")
#     print()
#     print()

In [303]:


X_train, X_test, y_train, y_test = train_test_split(
    Matr, y, test_size=0.4)

In [304]:

clf = LinearRegression(normalize=True)

In [305]:
clf.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [306]:
clf.score(X_test, y_test)

0.9544908240189351

In [307]:
list(zip(clf.predict(X_test), y_test))

[(147.69607569111787, 140.6),
 (-33.27395434523768, -11.73),
 (132.4870448842517, 131.34),
 (93.73308111154971, 91.85),
 (90.57649581431254, 86.064),
 (50.51408613262899, 49.741),
 (123.1443340287684, 125.665),
 (135.02957974059888, 132.69),
 (148.19813838840633, 143.0),
 (109.33020269773873, 113.467),
 (143.99390828661996, 142.48),
 (87.76597406513658, 89.784),
 (141.72896864470476, 142.0),
 (94.23514380883815, 98.427),
 (134.0073464526227, 133.8),
 (91.07855851160097, 93.475),
 (66.33834617182514, 68.74),
 (128.56964638647935, 126.54),
 (122.29426003919598, 111.969),
 (142.9706944466359, 137.68),
 (-42.07000000000001, -88.63),
 (150.16453608737544, 137.3),
 (42.276954345237485, 27.852),
 (150.5125473796594, 144.18),
 (122.64227133147996, 118.534),
 (77.90882107235356, 80.882),
 (122.79632273648441, 118.925),
 (52.13410733802051, 36.074),
 (115.60248109574489, 115.607),
 (53.32460788180494, 57.988)]

In [308]:
from sklearn.neighbors import KNeighborsRegressor

In [327]:
clf = KNeighborsRegressor(n_neighbors=3, weights="distance")

In [328]:
clf.fit(X_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=3, p=2,
          weights='distance')

In [329]:
clf.score(X_test, y_test)

0.8572906456378928